# Project 4: Supervised and Unsupervised Learning
## Predicting with Twitter Public API

### Starting the project

Test...

In [1]:
import numpy as np
import pandas as pd
import requests
import json

%matplotlib inline
import matplotlib.pyplot as plt

### Pubmed papers importing

Using Twitter API

### End